In [13]:
# from typing import Dict
# from code_keeper import cast_enum, parse_tagline
# from aenum import MultiValueEnum
# import uuid
# from dataclasses import dataclass
# from typing import List, Tuple, Any, Union
# from datetime import datetime
#
# import os
# import glob
# import json
# from collections import defaultdict
# class GardenArea(MultiValueEnum):
#     main = 1, 'main'
#     secondary = 2, 'secondary'
#     inbox = 3, 'inbox'
#
# @dataclass
# class CodeFragment:
#     name: str
#     code: str
#     categories: List[str]
#     tags: List[str]
#     area: GardenArea
#     id: str = None
#
#     def __post_init__(self):
#         if self.id is None:
#             self.id = str(uuid.uuid4())
#             # todo: _record_event - creation

class Singleton:
    pass

class CodeGarden(Singleton):
    def __init__(self, path):
        self.path = Path(path).expanduser()

    @staticmethod
    def _cast_area(area_path):
        return cast_enum(area_path, GardenArea)

    def _save(self, code, tags: List[str], area: GardenArea):
        # save to disk
        text = "\n".join(tags) + "\n" + code
        name = "_".join(tags) + '.py'
        area_path = Path(self.root, area.value)
        area_path.mkdir(parents=True, exist_ok=True)
        file_path = area_path / name
        with file_path.open("w") as f:
            f.write(text)

    def _find(self, keys: List[str]) -> Dict[str, CodeFragment]: # filepath -> code
        # option 1: load from disk
        # option 2: load on init and use from memory
        found_files = defaultdict(list)
        for key in keys:
            for area in GardenArea:
                area_path = Path(self.root, area.value)
                search_pattern = f"{area_path}/*{key}*.py"
                for file_path in glob.glob(search_pattern):
                    found_files[area].append(file_path)
        return found_files

    def _parse_keys(self, keys, sep='.') -> List[str]:
        if isinstance(keys, str):
            return [key.strip() for key in keys.split(sep)]

    def _parse_metadata(self, metadata_str: str) -> Dict[str, List[str]]:
        metadata = {}
        for part in metadata_str.split(";"):
            key, value = part.split("=")
            values = value.split(",")
            metadata[key.strip()] = values
        return metadata

    def _extract_tags_and_categories(self) -> Tuple[List[str], List[str]]:
        tags = set()
        categories = set()
        for area in GardenArea:
            area_path = Path(self.root, area.value)
            for file_path in area_path.glob("*.py"):
                with file_path.open("r") as f:
                    metadata_str = f.readline().strip()
                    metadata = self._parse_metadata(metadata_str)
                    tags.update(metadata["tags"])
                    categories.update(metadata["categories"])
        return list(tags), list(categories)


    def _generate_tags_summary(self) -> Dict[str, Any]:
        tags_summary = defaultdict(int)
        for area in GardenArea:
            area_path = Path(self.root, area.value)
            for file_path in area_path.glob("*.py"):
                with file_path.open("r") as f:
                    metadata_str = f.readline().strip()
                    metadata = self._parse_metadata(metadata_str)
                    for tag in metadata["tags"]:
                        tags_summary[tag] += 1
        return tags_summary

    def _generate_category_summary(self) -> Dict[str, Any]:
        category_summary = defaultdict(int)
        for area in GardenArea:
            area_path = Path(self.root, area.value)
            for file_path in area_path.glob("*.py"):
                with file_path.open("r") as f:
                    metadata_str = f.readline().strip()
                    metadata = self._parse_metadata(metadata_str)
                    for category in metadata["categories"]:
                        category_summary[category] += 1
        return category_summary

    def _record_event(self, event_type: str, code_fragment: CodeFragment):
        event = {
            "event_type": event_type,
            "code_fragment_id": code_fragment.id,
            "timestamp": datetime.now().isoformat()
        }
        events_file = Path(self.root, "events.json")
        if events_file.exists():
            with events_file.open("r") as f:
                events = json.load(f)
        else:
            events


    # def _suggest_metadata(self, code: str) -> Tuple[List[str], List[str]]:
    #     # Suggest metadata for the code fragment using AI


class CodeKeeper:
    # def __init__(self, code_garden: CodeGarden = None):
    #     if code_garden is None:
    #         code_garden = self.find_the_garden()
    #     elif isinstance(code_garden, str):
    #         code_garden = CodeGarden(code_garden)
    #     self.garden = code_garden


    # @staticmethod
    # def find_the_garden():
    #     root = os.environ.get("CODE_GARDEN_ROOT", "~/code_garden")
    #     return CodeGarden(root)

    def plant(self, code, tagline, area=GardenArea.inbox):
        tags_and_area = self._parse_tagline(tagline)
        tags = tags_and_area["tags"]
        if "area" in tags_and_area:
            area = parse_enum(tags_and_area["area"], GardenArea)
        self.garden._save(code, tags, area)

    def remind(self, keys=None):
        if keys is not None:
            if isinstance(keys, str):
                keys = [keys]

            # load all code patterns

            # if only one result -

    def remind(self, keys=None):
        if keys is None:
            keys = ['code_garden']
        elif isinstance(keys, str):
            keys = [keys]
            keys = self.garden._parse_keys(keys)
        found_files = self.garden._find(keys)
        result = []
        for area, files in found_files.items():
            for file in files:
                with open(file, "r") as f:
                    metadata_str = f.readline().strip()
                    code = f.read()
                result.append(f"Area: {area}\nFile: {file}\n{code}\n")
        return "\n".join(result)

    find = remind

    def generate_summary(self):
        tags_summary = self.garden._generate_tags_summary()
        category_summary = self.garden._generate_category_summary()
        return {"tags": tags_summary, "categories": category_summary}

    def _parse_tagline(self, tagline: str) -> Dict[str, Union[List[str], str]]:
        result = {"tags": []}
        for area in GardenArea:
            if tagline.startswith(area.value):
                result["area"] = area.value
                tagline = tagline[len(area.value) + 1:]
                break
        if tagline.endswith(".py"):
            tagline = tagline[:-3]
        tags = tagline.split("_")
        result["tags"].extend(tags)
        return result


MemoryKeeper = CodeKeeper


IndentationError: expected an indented block after function definition on line 176 (2718887312.py, line 179)

In [1]:
from pathlib import Path
p = "~/code_garden"
p = Path(p).expanduser()
ck = CodeKeeper(p)

NameError: name 'CodeKeeper' is not defined

['sample', 'tag', 'set', 'test', 'garden']


In [18]:
x = """/Users/calm/home/projects/venvs/py311/bin/python
/Users/calm/home/lib/calmlib/calmlib/utils/code_keeper/code_keeper
.py\n\n\t\b\h5yh9e8ordbiv `#G$^&%RI^QUEHAFDVBIRYSI%FU^OI&~*
Traceback (most recent call last):
  File "/Users/calm/home/lib/calmlib/calmlib/utils/code_keeper/code_keeper.py", line 12, in <module>
    from .utils import cast_enum
ImportError: attempted relative import with no known parent package"""
r = Path(x)
r.exists()

False

In [9]:
print(sorted([GardenArea.Main, GardenArea.Inbox, GardenArea.Secondary, ],key=lambda x: x.value))

[<GardenArea.Main: 1>, <GardenArea.Secondary: 2>, <GardenArea.Inbox: 3>]


In [11]:
# from enum import Enum
# class EE(Enum):
#     A = 1
#     B = 2
#
# sorted([EE.A, EE.B], key= lambda x: x.value)

[<EE.A: 1>, <EE.B: 2>]

In [ ]:
# ok, i need to play around with stuff


In [ ]:
#let's decide on a data model for this thing
# version 1:
# code, tags, area.

# version 2: + name, id, categories

In [ ]:
# let's go with version 1 for now..

In [ ]:
# option 1: do we load the data in the beginning
# option 2: or every time?

In [ ]:
# Version 3: Sync data in Notion

In [ ]:
# Version 4: track events and stats

In [19]:
# so, code_garden example:

from code_garden import CodeKeeper

ImportError: cannot import name 'CodeKeeper' from 'code_garden' (unknown location)

In [5]:
import code_garden

In [26]:
!pip install pyperclip

You should consider upgrading via the '/Users/calm/home/projects/venvs/py311/bin/python -m pip install --upgrade pip' command.


In [10]:
ck = code_garden.CodeKeeper()

AttributeError: module 'code_garden' has no attribute 'CodeKeeper'

In [1]:
import pyperclip

In [6]:
%reload code_garden

UsageError: Line magic function `%reload` not found.


In [7]:
from importlib import reload


In [15]:
import code_keeper

ModuleNotFoundError: No module named 'code_keeper'

In [16]:
import sys

In [17]:
print(sys.path)

['/Users/calm/Library/Application Support/JetBrains/Toolbox/apps/PyCharm-P/ch-0/231.8109.197/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug', '/Users/calm/Library/Application Support/JetBrains/Toolbox/apps/PyCharm-P/ch-0/231.8109.197/PyCharm.app/Contents/plugins/python/helpers/pydev', '/Users/calm/home/lib/calmlib/calmlib/utils/code_garden', '/Users/calm/home', '/Users/calm/home/lib/defaultenv', '/Users/calm/home/lib/magic_database', '/Users/calm/home/lib/gpt_api', '/Users/calm/home/projects/active/chatgpt_enhancer_bot', '/Users/calm/home/lib/bot_me_maybe', '/Users/calm/home/projects/active/chat_summary_bot', '/Users/calm/home/lib/calmlib', '/Users/calm/home/projects/playground/gpt-bot', '/Users/calm/code/anaconda3/envs/py311/lib/python311.zip', '/Users/calm/code/anaconda3/envs/py311/lib/python3.11', '/Users/calm/code/anaconda3/envs/py311/lib/python3.11/lib-dynload', '', '/Users/calm/home/projects/venvs/py311/lib/python3.11/site-packages', '/Users/calm/.local/lib/python3

In [19]:
from calmlib.utils.code_keeper import CodeKeeper

In [20]:
ck = CodeKeeper()
ck.generate_summary()

TypeError: unsupported operand type(s) for /: 'PosixPath' and 'int'